In [6]:
import pandas as pd

data = pd.read_csv("Desktop/Projects/incidents-model/data/raw/ITSM_data.csv")

C:\Users\thoma\AppData\Local\Temp\ipykernel_15960\1734445387.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Desktop/Projects/incidents-model/data/raw/ITSM_data.csv")


<h3>Data Sample</h3>

In [9]:
data.head(3)

,CI_Name,CI_Cat,CI_Subcat,WBS,Incident_ID,Status,Impact,Urgency,Priority,number_cnt,...,Reopen_Time,Resolved_Time,Close_Time,Handle_Time_hrs,Closure_Code,No_of_Related_Interactions,Related_Interaction,No_of_Related_Incidents,No_of_Related_Changes,Related_Change
0,SUB000508,subapplication,Web Based Application,WBS000162,IM0000004,Closed,4,4,4.0,0.601292,...,NaN,4/11/2013 13:50,4/11/2013 13:51,"3,87,16,91,111",Other,1.0,SD0000007,2.0,NaN,NaN
1,WBA000124,application,Web Based Application,WBS000088,IM0000005,Closed,3,3,3.0,0.415050,...,2/12/2013 12:31,2/12/2013 12:36,2/12/2013 12:36,"4,35,47,86,389",Software,1.0,SD0000011,1.0,NaN,NaN
2,DTA000024,application,Desktop Application,WBS000092,IM0000006,Closed,NS,3,NaN,0.517551,...,NaN,13-01-2014 15:12,13-01-2014 15:13,"4,84,31,19,444",No error - works as designed,1.0,SD0000017,NaN,NaN,NaN


<h3>Datatype by column</h3>

In [12]:
for column in data.columns:
    print(f'{column} is type: {type(column)}')

CI_Name is type: <class 'str'>
CI_Cat is type: <class 'str'>
CI_Subcat is type: <class 'str'>
WBS is type: <class 'str'>
Incident_ID is type: <class 'str'>
Status is type: <class 'str'>
Impact is type: <class 'str'>
Urgency is type: <class 'str'>
Priority is type: <class 'str'>
number_cnt is type: <class 'str'>
Category is type: <class 'str'>
KB_number is type: <class 'str'>
Alert_Status is type: <class 'str'>
No_of_Reassignments is type: <class 'str'>
Open_Time is type: <class 'str'>
Reopen_Time is type: <class 'str'>
Resolved_Time is type: <class 'str'>
Close_Time is type: <class 'str'>
Handle_Time_hrs is type: <class 'str'>
Closure_Code is type: <class 'str'>
No_of_Related_Interactions is type: <class 'str'>
Related_Interaction is type: <class 'str'>
No_of_Related_Incidents is type: <class 'str'>
No_of_Related_Changes is type: <class 'str'>
Related_Change is type: <class 'str'>
